In [1]:
import pandas as pd
import numpy as np

In [2]:
forecast_df = pd.read_csv("future_demand_forecast.csv")
residuals_df = pd.read_csv("demand_residuals.csv")

forecast = forecast_df["Forecast_Demand"].values
residuals = residuals_df["Residual"].values

n_days = len(forecast)

In [3]:
starting_inventory = 500
reorder_point = 200
reorder_qty = 400

# Supplier lead time (can randomize later)
lead_time = 7

# Monte Carlo settings
n_simulations = 10000

In [4]:
stockout_days_total = 0
avg_inventory_per_run = []

for sim in range(n_simulations):

    inventory = starting_inventory
    inventory_history = []

    pending_orders = []   # (arrival_day, qty)

    for day in range(n_days):

        # Receive arriving orders
        for order in pending_orders.copy():
            if order[0] == day:
                inventory += order[1]
                pending_orders.remove(order)

        # Generate stochastic demand
        random_error = np.random.choice(residuals)
        demand = forecast[day] + random_error

        demand = max(0, demand)

        # Consume inventory
        inventory -= demand

        if inventory <= 0:
            stockout_days_total += 1
            inventory = 0

        inventory_history.append(inventory)

        # Reorder logic
        if inventory <= reorder_point:
            arrival_day = day + lead_time
            pending_orders.append((arrival_day, reorder_qty))

    avg_inventory_per_run.append(np.mean(inventory_history))

In [6]:
total_days_simulated = n_simulations * n_days

stockout_probability = stockout_days_total / total_days_simulated
average_inventory = np.mean(avg_inventory_per_run)

print("Monte Carlo Inventory Simulation Results")
print("---------------------------------------")
print(f"Stockout Probability : {stockout_probability*100:.4f}%")
print(f"Average Inventory   : {average_inventory:.2f}")

Monte Carlo Inventory Simulation Results
---------------------------------------
Stockout Probability : 14.7738%
Average Inventory   : 945.28
